# Run Simulation

Test 25 is done after Test 24 had an error in the decimation counting. Test 24 is not useless, but it doesn't map the decimation properly. 

Below are the notes for Test 24 which apply nonetheless to Test 25.
In this notebook I run a large set of simulations for the dynamics of decimated colloidal ice. This will be stored under *Test 23*.

The purpose of *Test 23* is to understand the glassy behavior of undecimated colloidal ice. In all our previous simulations the correlation functions decay as a stretched exponential for several seconds after the quench is finished, and then the system is often frozen. This indicates a coarsening behavior before the system gets frozen. By applying a much slower quench, I hope to either equilibrate the system, or get it to a steady state within the quench time, so that the correlation function is observed on this final state. The question then will be what function is followed by the relaxation time. If it behaves like spin ice, this function will be a Vogel-Fulcher-Tammann. 

Each of these simulations consists of a square colloidal ice system, 
* decimated at a ratio $\xi$,
* starting from a random configuration, 
* quenched at a fixed rate $\dot{B}_q$, 
* until a final field is reached $B$,
and allowed to evolve at constant field for a long time. 

In [1]:
import sys
import os
import shutil
sys.path.insert(0, '../icenumerics/')
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import scipy.spatial as spa
import matplotlib.pyplot as plt
import matplotlib as mpl

import icenumerics as ice
from icenumerics.geometry import ordering 
import support as sp
import csv as csv
import time
import string as st
from multiprocessing import Pool

import copy as cp

ureg = ice.ureg

idx = pd.IndexSlice

import tqdm.notebook as tqdm

%reload_ext autoreload
%autoreload 2

/home/leogabac/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'support'

In [2]:
mpl.rc('text', usetex=True)
mpl.rcParams['figure.dpi'] = 150

In these simulations I map over several different values of the decimation, and over different values of final field. These are the parameters:

In [3]:
n_of_fields = 10 # 10
n_of_experiments = 1 # 1
n_of_decimations = 10 # 10
montecarlo = 25000 # 1e4 
total_time = 1000*ureg.s # 3600 sec
framerate = 3*ureg.Hz
timestep = 1e-3*ureg.s
processors = 4

These are geometric parameters. Particles are $2.8\mu{}m$ Dynabeads, and traps are separated by $3\mu{}m$.

In [4]:
rho = 1.4*ureg.um
t = 3*ureg.um
lattice = np.sqrt(2)*(2*rho+1*ureg.um)+t
lattice_size = [100,100] # 20,20

This is the array of fields to be used

In [5]:
fieldrate = 2/300 #mT/sec

max_fields = np.linspace(0.5,2,n_of_fields)

qts = max_fields/fieldrate

qts

array([ 75., 100., 125., 150., 175., 200., 225., 250., 275., 300.])

In [6]:
fields = qts*fieldrate
fields

array([0.5       , 0.66666667, 0.83333333, 1.        , 1.16666667,
       1.33333333, 1.5       , 1.66666667, 1.83333333, 2.        ])

Experiments are repetitions with identical parameters

In [7]:
exp = np.arange(n_of_experiments)
exp

array([0])

In [8]:
def how_many_traps_to_remove(chis):
    # N is the number of vertices. 
    N = lattice_size[0]*lattice_size[1]
    # T is the number of traps
    T = 2*N

    # N3 is the number of decimated vertices
    N3 = chis*N
    # for each decimated vertex, we must remove half a trap
    Tr = N3/2

    return np.round(Tr)

Decimation arrays

In [9]:
max_chi = 0.25
chis = np.linspace(0,max_chi,n_of_decimations)

n_to_remove = how_many_traps_to_remove(chis)

chis

array([0.        , 0.02777778, 0.05555556, 0.08333333, 0.11111111,
       0.13888889, 0.16666667, 0.19444444, 0.22222222, 0.25      ])

In [10]:
n_to_remove

array([   0.,  139.,  278.,  417.,  556.,  694.,  833.,  972., 1111.,
       1250.])

In [11]:
exps = np.meshgrid(exp,qts,chis)

for i in range(len(exps)):
    exps[i] = exps[i].flatten()
exps = np.array(exps).transpose()

In [12]:
len(exps)

100

In [13]:
directory = "/home/aortiza/Data/DecimatedSquare/test25"
test_name = "decimation"

In [14]:
try:
    shutil.rmtree(directory)
except Exception as e:
    print(e)
    
if not os.path.exists(directory):
    os.makedirs(directory)

In [15]:
field_template = st.Template("v_Bmag/$Qt*time/1e6*((time/1e6)<$Qt)+v_Bmag*((time/1e6)>=$Qt)")
field_template.substitute(Qt = qts[len(qts)-1])

'v_Bmag/300.0*time/1e6*((time/1e6)<300.0)+v_Bmag*((time/1e6)>=300.0)'

In [16]:
particle = ice.particle(radius = 1.4*ureg.um,
         susceptibility = 0.4,
         diffusion = 0.14*ureg.um**2/ureg.s,
         temperature = 300*ureg.K,
         density = 1000*ureg.kg/ureg.m**3)

trap = ice.trap(trap_sep = 3*ureg.um,
               height = 8*ureg.pN*ureg.nm, # the stiffness is around 2 KbT. 
               stiffness = 100e-6*ureg.pN/ureg.nm, #100e-6
               cutoff=np.Inf*ureg.um)

In [17]:
spins = ice.spins()
spins.create_lattice("square",lattice_size, lattice_constant = lattice, border = "periodic")
spins.order_spins(ordering.random_ordering)

In [ ]:
graph_base = sp.graph()
graph_base = graph_base.spins_to_graph(spins, periodic = True,
                         region = np.array(spins.size+[np.inf])*spins.lattice.magnitude)

remove = {e: sp.create_decimation(graph_base, iterations = montecarlo) for e in tqdm.tqdm(exp)}

For these large arrays, creating a decimation might take a few hours. We will save them as a dataset to be reused if something goes wrong.

In [ ]:
for e in exp:
    remove[e].to_csv(os.path.join(directory,"remove_list_%u.dat"%e),sep="\t")

In [ ]:
def create_decimations(graph_base,remove,chi):
    """ Creates an array of decimations.
    A decimation is a list containing the (position of) traps which, if removed from a full system, 
    generate a structure with decimation chi.
    --------
    Parameters
    
    graph_base:
    remove:
    chi:
    iterations:
    """    
    centers = graph_base.spins()[0][remove.index]
    remove_no = np.int64(how_many_traps_to_remove(chi))
    remove_no = remove_no[remove_no<=len(centers)]
    
    try: 
        return centers[0:remove_no]
        
    except TypeError:
        return [centers[:r] for r in remove_no]

In [ ]:
col = ice.colloidal_ice(spins, particle, trap, height_spread = 0, susceptibility_spread = 0.1)
col.region[:,:2]=(np.array([np.array([0,0]),lattice_size])-0.1)*lattice
col.region[:,2] = np.array([-.11,.11])*ureg.um

def create_col(exp,remove):
    """ Create a col object with the parameters (exp, qt, chi) given by exp"""

    exp_no, qt, chi = tuple(exp)

    col_dec = col.copy(deep=True)
    
    decimation = create_decimations(cp.deepcopy(graph_base), remove[exp_no], chi)
    # if the montecarlo didn't produce enough removable traps to fulfill a decimation, it is empty. 
    # The function returns none.
    if len(decimation)>0:
    
        to_remove = [col_dec.where(d*ureg.um)[0] for d in decimation[0]]
        col_dec.remove(to_remove)

        Nz3 = 2*(len(col)-len(col_dec))
        N = len(col)/2
        chi = Nz3/N

        world = ice.world(
            field = fieldrate*(qt)*ureg.mT,
            temperature = 300*ureg.K,
            dipole_cutoff = 40*ureg.um,
            enforce2d = True,
            boundaries = ["p", "p", "p"])

        col_dec.simulation(world,
                 name = test_name+"chi_%f_qt_%2.2f_exp_%u"%(chi,qt,exp_no),
                 include_timestamp = False,
                 targetdir = directory,
                 framerate = framerate,
                 timestep = timestep,
                 run_time = total_time,
                 output = ["x","y","z", "mux", "muy", "muz"],
                 processors = processors)

        col_dec.sim.field.fieldz = field_template.substitute(Qt = qt)

        return col_dec

In [ ]:
def run_col(exp):
    """
    Runs the simulation defined by a Colloidal Ice object. 
    This also converts the lammpstrj to a trj array and writes a line in the index.dat, with the parameters. 
    """
    col = create_col(exp,remove)

    if col is not None:
        
        exp_no, qt, chi = tuple(exp)

        col.randomize()
        col.run_simulation()
        name = os.path.split(col.sim.base_name)[1]
        try: 
            sp.get_ice_trj_low_memory_hdf(col)

            with open(os.path.join(directory,"index.dat"),'a',newline='') as file:
                writer = csv.writer(file,delimiter='\t')
                writer.writerow([name, chi, qt, exp_no])

        except FileNotFoundError:
            print("filenotfound")
            pass

In [ ]:
from multiprocessing import Pool

In [ ]:
%%time
if __name__ ==  '__main__': 
    num_processors = 4
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(os.path.join(directory,"index.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "chi", "qt", "exp"])
    p=Pool(processes = num_processors)
    list(tqdm.tqdm(p.imap(run_col, exps), total=len(exps)))